In [1]:
import pandas as pd
import os
import pyodbc
import sqlalchemy
import carbon_tracker as ct
if not os.path.exists('./emissions'):
	os.makedirs('./emissions')
ct.start_tracker() 

[codecarbon INFO @ 15:35:37] offline tracker init
[codecarbon WARNING @ 15:35:37] Multiple instances of codecarbon are allowed to run at the same time.


In [2]:
ct.update_project_name('Connect to SQL SERVER')

In [3]:
pyodbc.drivers()

['SQL Server',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

In [4]:
from dotenv import load_dotenv

load_dotenv()

connection_string = os.getenv('DB_CONNECTION_STRING')
conn = pyodbc.connect(connection_string)
print("Connection Successful")


Connection Successful


In [5]:
ct.update_project_name('Load csv into SQL Server')

In [6]:
import csv

# Get connection from environment variable
connection_string = os.getenv('DB_CONNECTION_STRING')
connection = pyodbc.connect(connection_string)

try:
    with open('DataSources/orders.csv', 'r') as f:
        reader = csv.reader(f)
        columns = next(reader)

        clean_columns = [c.strip().replace(' ', '_') for c in columns]
        
        table_name = "PythonETL.dbo.orders"  
 
        create_query = f"IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'orders' AND schema_id = SCHEMA_ID('dbo') AND OBJECT_ID IN (SELECT OBJECT_ID FROM sys.objects WHERE type = 'U' AND DB_NAME() = 'PythonETL')) BEGIN CREATE TABLE {table_name} ({', '.join([f'{col} VARCHAR(255)' for col in clean_columns])}) END"
        
        cursor = connection.cursor()
        cursor.execute(create_query)
        connection.commit()
        
        # Insert the data
        insert_query = 'INSERT INTO {0}({1}) VALUES ({2})'
        insert_query = insert_query.format(
            table_name,
            ','.join(clean_columns),
            ','.join('?' * len(columns))
        )
        
        count = 0
        for data in reader:
            cursor.execute(insert_query, data)
            count += 1
            
            if count % 1000 == 0:
                connection.commit()
                print(f"Inserted {count} rows...")
        
        connection.commit()
        print(f"Successfully inserted {count} rows into {table_name}")
        
except Exception as e:
    print(f"Error: {str(e)}")
    connection.rollback()
finally:
    cursor.close()
    connection.close()

Inserted 1000 rows...
Successfully inserted 1615 rows into PythonETL.dbo.orders


In [7]:
ct.update_project_name('Get inserted data as dataframe')

In [8]:
query = """
SELECT *
FROM PythonETL.dbo.orders
"""

orders = pd.read_sql(query, conn)
orders.head()


C:\Users\oo\AppData\Local\Temp\ipykernel_22568\2927504220.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql(query, conn)


,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6


In [10]:
conn.close()

In [12]:
ct.stop_tracker()
ct.show_metrics()

Total Emissions per Project:

Project Name: Python ETL Pipeline
Total Emissions: 0.00 kg CO2
Run Duration: 0 days, 00 hours, 00 minutes and 19 seconds
Total Duration: 0 days, 00 hours, 00 minutes and 00 seconds
Energy Consumed: 0.00 kWh
CPU Power: 60.00 W
GPU Power: 0.00 W
RAM Power: 10.00 W

Travel Equivalences:
  - driving an electric car: 0.0002 km
  - traveling by TGV: 0.0089 km

Production Equivalences:

Consumption Equivalences:
  - hours of video streaming: 0.0003

Project Name: Python ETL Pipeline Connect to SQL SERVER
Total Emissions: 0.00 kg CO2
Run Duration: 0 days, 00 hours, 00 minutes and 28 seconds
Total Duration: 0 days, 00 hours, 00 minutes and 00 seconds
Energy Consumed: 0.00 kWh
CPU Power: 60.00 W
GPU Power: 0.00 W
RAM Power: 10.00 W

Travel Equivalences:
  - driving a thermal car: 0.0001 km
  - flying by plane: 0.0001 km
  - driving an electric car: 0.0003 km
  - traveling by TGV: 0.0133 km

Production Equivalences:

Consumption Equivalences:
  - hours of video strea